# Creación de grafos
Funcionalidades para transformar los archivos ".csv" dados en grafos guardados en archivos con la extensión ".gexf".

Dos tipos de modos: 
* MODE = "user": red de usuarios unidos si han usado uno (o más) hashtags
* MODE = "hashtag": red de hashtags unidos si han sido usados por uno (o más) usuarios

In [50]:
import networkx as nx
import pandas as pd
from tqdm import tqdm
import os

In [51]:
data_folder = "data/csv/"
graphs_folder = "graphs/"
plots_folder = "plots/"

# Seleccionamos el fichero correspondiente a la manifestación social de la que queremos sacar la red
MANIFESTACION = "9n"
#MANIFESTACION = "nat"

# Seleccionamos el modo con el que queremos crear nuestra red: Hashtags como nodos o usuarios como nodos
#MODE = "user"
MODE = "hashtag"


In [52]:
def select_file(MANIFESTACION):
    files = os.listdir(data_folder + MANIFESTACION + '/')

    # Solo hay un archivo en el directorio
    if len(files) == 1:
        name_file = files[0]
        file_selected = os.path.join(data_folder + MANIFESTACION + '/', name_file)
        return file_selected
    else:
        print("ERROR: l directorio no contiene un solo archivo.")
        return -1

In [53]:
file_selected = select_file(MANIFESTACION)
modes_folder = "nodes_" + MODE + '/'
graphs_folder = graphs_folder + modes_folder + MANIFESTACION + '/'
plots_folder = plots_folder + modes_folder + MANIFESTACION + '/'

In [54]:
df = pd.read_csv(file_selected, sep=' ')
df_h = df["hour"].unique()

In [55]:
def create_graphs(node_criteria, edge_criteria):
    print("Creando redes de", node_criteria, "unidos si comparten uno o más", edge_criteria, ", manifestación seleccionada:", MANIFESTACION)
    G = nx.Graph()
    for hour in tqdm(df_h):
        df_hour = df[(df["hour"] == hour)]
        df_nodes = df_hour[node_criteria].unique()
        G.add_nodes_from(df_nodes)
        for node in df_nodes:
            # Seleccionamos las filas del dataframe con el usuario/hashtag sobre el que iteramos
            df_node_edge = df_hour.loc[df_hour[node_criteria] == node]
            # Seleccionamos tantos hashtags/usuarios como haya que haya compartido el usuario/hasthag respectivamente
            df_node_edge = df_node_edge[edge_criteria]
            for edge in df_node_edge:
                df_edge = df_hour.loc[df_hour[edge_criteria] == edge]
                df_edge = df_edge[node_criteria]
                for nd in df_edge:
                    if nd != node:
                        if G.has_edge(node, nd):
                            G[node][nd]["weight"] += 1
                        else:
                            G.add_edge(node, nd, weight = 1)
        nx.write_gexf(G, graphs_folder + str(hour) + ".gexf")

In [56]:
if MODE == "user":
    create_graphs("user", "hashtag")
elif MODE == "hashtag":
    create_graphs("hashtag", "user")
else:
    print("ERROR: MODE solo puede ser de tipo user o hashtag")

Creando redes de hashtag unidos si comparten uno o más user , manifestación seleccionada: 9n


100%|██████████| 97/97 [06:26<00:00,  3.98s/it]
